In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import data
import lightgbm as lgb
import numpy as np
import os
import sys
import pandas as pd
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score
from gensim.models import word2vec
import logging

from model import lgb_model

%matplotlib inline

In [2]:
train_ad, train_click, train_user, test_ad, test_click = data.load_data()
train_user, valid_user = train_test_split(train_user, test_size=0.33, random_state=42)
train_record = data.get_part_click(train_click, train_user)
valid_record = data.get_part_click(train_click, valid_user)

# train_record
train_record = pd.merge(train_record, train_ad, on="creative_id")
# valid_record
valid_record = pd.merge(valid_record, train_ad, on="creative_id")

In [3]:
train_features, train_age, train_gender = data.split_feature_target(train_record, keep_user=True)
valid_features, valid_age, valid_gender = data.split_feature_target(valid_record, keep_user=True)

In [4]:
train_features.head()

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,6,309204,325532,1,292523,27081.0,3,32066,242.0
1,5,801758,325532,1,292523,27081.0,3,32066,242.0
2,5,126648,325532,1,292523,27081.0,3,32066,242.0
3,59,309204,2746730,1,2362208,NaN,18,14682,88.0
4,68,317233,2746730,1,2362208,NaN,18,14682,88.0


In [9]:
grouped_record = train_features.groupby("user_id")

In [20]:
for user_id, record in grouped_record:
    print(user_id)
    record = record.sort_values(by="time")
    sentence = list(record["creative_id"])
    print(record)
    print(sentence)
    break

2
          time  user_id  creative_id  click_times    ad_id  product_id  \
3398990     10        2        63441            1    58788        87.0   
18079728    11        2       155822            1   139702        80.0   
1617395     14        2        39714            1    38066       129.0   
3529924     17        2       609050            1   541125       129.0   
272400      28        2        13069            1    14495      1400.0   
1242952     28        2      1266180            1  1107111         NaN   
2241262     28        2       441462            1   392680        87.0   
7302729     38        2      1657530            1  1436687      1261.0   
1861396     38        2      1696925            1  1469873         NaN   
7291835     39        2       769749            1   680028         NaN   
9265604     41        2      1074235            1   942883       111.0   
4230871     42        2      1252062            1  1095314       129.0   
6261336     42        2      1662244

In [ ]:
def word_embedding(intput_record):
    

In [15]:
#删掉user_id, 把age和gender当成label
'''
features中各列的含义
1. time
2. creative_id
3. click_times
4. ad_id
5. product_id
6. product_category
7. advertiser_id
8. industry_id
'''
train_features, train_age, train_gender = data.split_feature_target(train_record)
valid_features, valid_age, valid_gender = data.split_feature_target(valid_record)

train_features = train_features.values
train_age = train_age.values - 1
train_gender = train_gender.values - 1

valid_features = valid_features.values
valid_age = valid_age.values - 1
valid_gender = valid_gender.values - 1

In [16]:
train_features[:5]

array([[6.000000e+00, 3.255320e+05, 1.000000e+00, 2.925230e+05,
        2.708100e+04, 3.000000e+00, 3.206600e+04, 2.420000e+02],
       [5.000000e+00, 3.255320e+05, 1.000000e+00, 2.925230e+05,
        2.708100e+04, 3.000000e+00, 3.206600e+04, 2.420000e+02],
       [5.000000e+00, 3.255320e+05, 1.000000e+00, 2.925230e+05,
        2.708100e+04, 3.000000e+00, 3.206600e+04, 2.420000e+02],
       [5.900000e+01, 2.746730e+06, 1.000000e+00, 2.362208e+06,
                 nan, 1.800000e+01, 1.468200e+04, 8.800000e+01],
       [6.800000e+01, 2.746730e+06, 1.000000e+00, 2.362208e+06,
                 nan, 1.800000e+01, 1.468200e+04, 8.800000e+01]])

In [21]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# 引入数据集
raw_sentences = ["the quick brown fox jumps over the lazy dogs","yoyoyo you go home now to sleep"]

# 切分词汇
sentences= [s.split() for s in raw_sentences]

In [22]:
sentences

[['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dogs'],
 ['yoyoyo', 'you', 'go', 'home', 'now', 'to', 'sleep']]

In [23]:
# 构建模型
model = word2vec.Word2Vec(sentences, min_count=1)

2020-05-30 15:16:37,285 : INFO : collecting all words and their counts
2020-05-30 15:16:37,286 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-30 15:16:37,286 : INFO : collected 15 word types from a corpus of 16 raw words and 2 sentences
2020-05-30 15:16:37,287 : INFO : Loading a fresh vocabulary
2020-05-30 15:16:37,287 : INFO : effective_min_count=1 retains 15 unique words (100% of original 15, drops 0)
2020-05-30 15:16:37,288 : INFO : effective_min_count=1 leaves 16 word corpus (100% of original 16, drops 0)
2020-05-30 15:16:37,288 : INFO : deleting the raw counts dictionary of 15 items
2020-05-30 15:16:37,289 : INFO : sample=0.001 downsamples 15 most-common words
2020-05-30 15:16:37,289 : INFO : downsampling leaves estimated 2 word corpus (13.7% of prior 16)
2020-05-30 15:16:37,290 : INFO : estimated required memory for 15 words and 100 dimensions: 19500 bytes
2020-05-30 15:16:37,290 : INFO : resetting layer weights
2020-05-30 15:16:37,293 : INFO :

In [20]:
a = model['dogs']
a.min()
a.max()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


-0.004990018

0.0048909504